# Drop Volume if exists

In [0]:
%sql
DROP VOLUME IF EXISTS workspace.bronze_pyspark.source_crm

In [0]:
%sql
DROP VOLUME IF EXISTS workspace.bronze_pyspark.source_erp

# If Tables are already exists, drop them

In [0]:
tables = ['workspace.bronze_pyspark.crm_customers',
                         'workspace.bronze_pyspark.crm_products',
                         'workspace.bronze_pyspark.crm_sales',
                         'workspace.bronze_pyspark.erp_cust_az12',
                         'workspace.bronze_pyspark.erp_loc_a101',
                         'workspace.bronze_pyspark.erp_px_cat']
for table in tables:
    f'DROP TABLE IF EXISTS {table}'

# Create a dict for ingestion_path

In [0]:
base_path = '/Volumes/workspace/bronze_pyspark/source_systems'

ingetion_path = [
    #crm
    {'source': 'crm',
    'path': f'{base_path}/source_crm/cust_info.csv',
    'dest_table': 'workspace.bronze_pyspark.crm_customers'},
    {'source': 'crm',
    'path': f'{base_path}/source_crm/prd_info.csv',
    'dest_table': 'workspace.bronze_pyspark.crm_products'},
    {'source': 'crm',
    'path': f'{base_path}/source_crm/sales_details.csv',
    'dest_table': 'workspace.bronze_pyspark.crm_sales'},
    #erp
    {'source': 'erp',
    'path': f'{base_path}/source_erp/CUST_AZ12.csv',
    'dest_table': 'workspace.bronze_pyspark.erp_cust_az12'},
    {'source': 'erp',
    'path': f'{base_path}/source_erp/LOC_A101.csv',
    'dest_table': 'workspace.bronze_pyspark.erp_loc_a101'},
    {'source': 'erp',
    'path': f'{base_path}/source_erp/PX_CAT_G1V2.csv',
    'dest_table': 'workspace.bronze_pyspark.erp_px_cat'}
    ]

# import methods & libraries 

In [0]:
from pyspark.sql import functions as f
from pyspark.sql.types import *
import random


# Read the CSV from volumes & ingest it to tables

In [0]:
for item in ingetion_path:
    df = spark.read.option('header', 'true').option('inferschema','true').csv(item['path'])
    df.write.mode('overwrite').saveAsTable(item['dest_table'])
    display(df.limit(5)) # pyspark
    #spark.sql(f"select * from {item['dest_table']} LIMIT 5").show() # pyspark sql

